In [25]:
import datetime
from time import sleep, time

from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import requests
from bs4 import BeautifulSoup
import csv
from pathlib import Path

from lxml import etree

In [61]:
filename = "articles_info.csv" # имя файла, в который будем сохранять результат
driver_path = r"C:\Users\User\Downloads\chromedriver_win32\chromedriver.exe"
base_dir= r"C:\Users\User\Desktop" # укажите директорию, в которую будем сохранять файл
user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.81 Safari/537.36" #Ваш user-agent. Узнать можно тут https://юзерагент.рф, смотреть с браузера Chrome
start_time = time() # время начала выполнения программы

In [4]:
def get_load_time(article_url, user_agent):
    #будем ждать 3 секунды, иначе выводить exception и присваивать константное значение
    try:
        # меняем значение заголовка. По умолчанию указано, что это python-код
        headers = {
            "User-Agent": user_agent
        }
        # делаем запрос по url статьи article_url
        response = requests.get(
            article_url, headers=headers, stream=True, timeout=3.000
        )
        # получаем время загрузки страницы
        load_time = response.elapsed.total_seconds()
    except Exception as e:
        print(e)
        load_time = ">3"
    return load_time

In [15]:
def write_to_file(output_list, filename, base_dir):
    for row in output_list:
        with open(Path(base_dir).joinpath(filename), "a") as csvfile:
            fieldnames = ["id", "load_time", "rank", "points", "title", "url", "num_comments"]
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            writer.writerow(row)

In [6]:
def connect_to_base(browser, page_number):
    base_url = "https://news.ycombinator.com/news?p={}".format(page_number)
    for connection_attempts in range(1,4): # совершаем 3 попытки подключения
        try:
            browser.get(base_url)
            # ожидаем пока элемент table с id = 'hnmain' будет загружен на страницу
            # затем функция вернет True иначе False 
            WebDriverWait(browser, 5).until(
                EC.presence_of_element_located((By.ID, "hnmain"))
            )
            return True
        except Exception as e:
            print(e)
            print("Error connecting to {}.".format(base_url))
            print("Attempt #{}.".format(connection_attempts))
    return False

In [59]:
def parse_html(html, user_agent):
    soup = BeautifulSoup(html, "html.parser")
    output_list = []
    
    # ищем в объекте soup object id, rank, score и title статьи
    tr_blocks = soup.find_all("tr", class_="athing")
    article = 0
    
    # начальный номер строки, где хранится дополнительная инфа (subtext)
    subtext_row = 2 
    
    # понадобится для использования xpath, чтобы найти количество комментариев
    dom = etree.HTML(str(soup))
    
    for tr in tr_blocks:
        article_id = tr.get("id") # id
        article_url = tr.find_all("a")[1]["href"]
        
        # иногда статья располагается не на внешнем сайте, а на ycombinator, тогда article_url у нее не полный, 
        # а добавочный, с параметрами. Например item?id=200933. Для этих случаев будем добавлять урл до полного
        if "item?id=" in article_url or "from?site=" in article_url:
            article_url = f"https://news.ycombinator.com/{article_url}"
            
        load_time = get_load_time(article_url, user_agent)
        
        # иногда рейтинга может не быть, поэтому воспользуемся try
        try:
            score = soup.find(id=f"score_{article_id}").string
        except Exception as e:
            print(e)
            score = "0 points"
        
        # иногда комментариев может не быть, поэтому воспользуемся try
        try:
            #print(f'//*[@id="hnmain"]/tbody/tr[3]/td/table/tbody/tr[{subtext_row}]/td[2]/a[3]')
            num_comments = dom.xpath(f'//*[@id="hnmain"]/tbody/tr[3]/td/table/tbody/tr[{subtext_row}]/td[2]/a[3]')[0].text
        except Exception as e:
            print(e)
            num_comments = "0 comments"    
        # следующий номер строки, где хранится дополнительная инфа (subtext)
        subtext_row += 3

        article_info = {
            "id": article_id,
            "load_time": load_time,
            "rank": tr.span.string,
            "points": score,
            "title": tr.find(class_="storylink").string,
            "url": article_url,
            "num_comments": num_comments
        }

        # добавляем информацию о статье в список
        output_list.append(article_info)
        article += 1
    return output_list

In [62]:
# инициализируем веб драйвер
browser = webdriver.Chrome(executable_path=driver_path)

# перебираем страницы и собираем нужную информацию
for page_number in range(10):
    print("getting page " + str(page_number) + "...")
    if connect_to_base(browser, page_number):
        sleep(5)
        output_list = parse_html(browser.page_source, user_agent)
        write_to_file(output_list, filename, base_dir)

    else:
        print("Error connecting to hacker news")
    
# завершаем работу драйвера
browser.close()
sleep(1)
browser.quit()
end_time = time()
elapsed_time = end_time - start_time
print("run time: {} seconds".format(elapsed_time))

getting page 0...


KeyboardInterrupt: 

In [63]:
from concurrent.futures import ThreadPoolExecutor, wait

def run_process(page_number, filename):
    browser = webdriver.Chrome(executable_path=driver_path)
    if connect_to_base(browser, page_number):
        sleep(5)
        output_list = parse_html(browser.page_source, user_agent)
        write_to_file(output_list, filename, base_dir)
        
        browser.quit()
    else:
        print("Error connecting to hacker news")
        browser.quit()


In [64]:
futures = []

with ThreadPoolExecutor() as executor:
    for number in range(10):
        futures.append(
            executor.submit(run_process, number, filename)
        )
wait(futures)
end_time = time()
elapsed_time = end_time - start_time
print("Elapsed run time: {} seconds".format(elapsed_time))

Message: 

Error connecting to https://news.ycombinator.com/news?p=4.
Attempt #1.
Message: 

Error connecting to https://news.ycombinator.com/news?p=6.
Attempt #1.
Message: 

Error connecting to https://news.ycombinator.com/news?p=7.
Attempt #1.
Message: 

Error connecting to https://news.ycombinator.com/news?p=9.
Attempt #1.
Message: 

Error connecting to https://news.ycombinator.com/news?p=8.
Attempt #1.
'NoneType' object has no attribute 'string'
list index out of range
'NoneType' object has no attribute 'string'
list index out of range
Message: 

Error connecting to https://news.ycombinator.com/news?p=8.
Attempt #2.
HTTPSConnectionPool(host='passionhacks.com', port=443): Max retries exceeded with url: /gradient-isometric-illustrations/ (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1123)')))
HTTPSConnectionPool(host='ruqqus.com', port=443): Read timed out. (read timeout=3.0)